In [1]:
!pip install transformers

     |████████████████████████████████| 769 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 660 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 32.6 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=a3e53a7010b038898e2c3ad0dc31c25f0d5f0cd722549f945fdf4a19bc4a349e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


### BERT 

In [1]:
from transformers import BertConfig
from transformers import BertTokenizer
from transformers import BertForMaskedLM
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

model = 'bert-base-chinese'
config = BertConfig.from_pretrained(model)
tokenizer = AutoTokenizer.from_pretrained(model)

NUM_RESULTS = 3  
model = AutoModelWithLMHead.from_pretrained(model)
model.eval() 

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/modeling_auto.py:798: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequen

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

### wwm BERT

In [ ]:
model2 = 'hfl/chinese-bert-wwm-ext'
config = BertConfig.from_pretrained(model2)
tokenizer = AutoTokenizer.from_pretrained(model2)

NUM_RESULTS = 10  
model2 = AutoModelWithLMHead.from_pretrained(model2)
model2.eval() 

In [2]:
import copy 
def predict(text, model, num_results):
    if isinstance(text, str):
        tokenized_text = tokenizer.tokenize(text)
    else:
        tokenized_text = text 
    mask_ix = tokenized_text.index('[MASK]') 
    input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokenized_text)).unsqueeze(0)
    mask_distribution = model(input_ids)[0][:, mask_ix]
    topk_probs, topk_vals = mask_distribution.topk(num_results)
    return topk_probs, topk_vals, tokenized_text 

def predict_masks(text, choices, masks_left, num_results, model=model):
    if masks_left < 1:
        return 
    topk_probs, topk_vals, tokenized_text = predict(text, model, num_results)
    if masks_left == 1:
        for i in range(num_results):
            token_id = int(topk_vals[0][i])
            prob = float(topk_probs[0][i])
            token = tokenizer._convert_id_to_token(token_id)
            print(token, prob)
        print() 
        return 
    for j in range(num_results):
        token_id = int(topk_vals[0][j])
        prob = float(topk_probs[0][j])
        token = tokenizer._convert_id_to_token(token_id)
        updated_text = copy.deepcopy(tokenized_text)
        print(token, prob)
        updated_text[updated_text.index('[MASK]')] = token 
        print(updated_text)
        predict_masks(updated_text, choices, masks_left-1, num_results, model=model)

In [13]:
TEXT = """
黑夜给了我黑色的眼睛 我却用它寻找[MASK][MASK]。
"""

CHOICES = [] 

predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model2)

自 8.543882369995117
['黑', '夜', '给', '了', '我', '黑', '色', '的', '眼', '睛', '我', '却', '用', '它', '寻', '找', '自', '[MASK]', '。']
己 15.226829528808594
由 12.742009162902832
我 12.25773811340332

生 8.116238594055176
['黑', '夜', '给', '了', '我', '黑', '色', '的', '眼', '睛', '我', '却', '用', '它', '寻', '找', '生', '[MASK]', '。']
活 13.305920600891113
命 12.86294937133789
存 10.92110538482666

一 7.781883716583252
['黑', '夜', '给', '了', '我', '黑', '色', '的', '眼', '睛', '我', '却', '用', '它', '寻', '找', '一', '[MASK]', '。']
切 11.58308219909668
生 10.99929141998291
年 9.165112495422363

------------------------------------------------------------------------------------
时 8.857612609863281
['黑', '夜', '给', '了', '我', '黑', '色', '的', '眼', '睛', '我', '却', '用', '它', '寻', '找', '时', '[MASK]', '。']
刻 18.439910888671875
间 17.01576805114746
代 14.826703071594238

爱 8.689260482788086
['黑', '夜', '给', '了', '我', '黑', '色', '的', '眼', '睛', '我', '却', '用', '它', '寻', '找', '爱', '[MASK]', '。']
情 16.697908401489258
间 12.10219955444336
心 11.526802062988281

In [14]:
TEXT = """
《成见》
即使套上垃圾袋
也不会有人
觉得垃圾桶很[MASK][MASK]
"""

CHOICES = []
predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model2)

有 10.387682914733887
['《', '成', '见', '》', '即', '使', '套', '上', '垃', '圾', '袋', '也', '不', '会', '有', '人', '觉', '得', '垃', '圾', '桶', '很', '有', '[MASK]']
用 11.291385650634766
名 10.417311668395996
效 10.158729553222656

重 9.647831916809082
['《', '成', '见', '》', '即', '使', '套', '上', '垃', '圾', '袋', '也', '不', '会', '有', '人', '觉', '得', '垃', '圾', '桶', '很', '重', '[MASK]']
的 8.205307006835938
要 8.179563522338867
大 8.116860389709473

大 9.458673477172852
['《', '成', '见', '》', '即', '使', '套', '上', '垃', '圾', '袋', '也', '不', '会', '有', '人', '觉', '得', '垃', '圾', '桶', '很', '大', '[MASK]']
。 8.153440475463867
的 7.610923767089844
... 7.576132297515869

------------------------------------------------------------------------------------
肮 10.73465633392334
['《', '成', '见', '》', '即', '使', '套', '上', '垃', '圾', '袋', '也', '不', '会', '有', '人', '觉', '得', '垃', '圾', '桶', '很', '肮', '[MASK]']
脏 20.972110748291016
净 17.038986206054688
肮 16.17559242248535

脏 10.649730682373047
['《', '成', '见', '》', '即', '使', '套', '上', '垃', '圾', '袋', '也

In [15]:
TEXT = """
《说服》
你是冰
我是[MASK][MASK]
"""

CHOICES = []

predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model2)

王 6.742694854736328
['《', '说', '服', '》', '你', '是', '冰', '我', '是', '王', '[MASK]']
者 7.674739837646484
， 6.377195358276367
子 6.348962783813477

马 6.220652103424072
['《', '说', '服', '》', '你', '是', '冰', '我', '是', '马', '[MASK]']
（ 7.140013217926025
儿 6.351709365844727
， 6.261220455169678

台 5.686470031738281
['《', '说', '服', '》', '你', '是', '冰', '我', '是', '台', '[MASK]']
湾 8.377700805664062
马 6.711005210876465
小 6.70683479309082

------------------------------------------------------------------------------------
中 8.277717590332031
['《', '说', '服', '》', '你', '是', '冰', '我', '是', '中', '[MASK]']
国 10.422002792358398
文 8.68859577178955
华 8.126713752746582

游 7.909826278686523
['《', '说', '服', '》', '你', '是', '冰', '我', '是', '游', '[MASK]']
戏 17.488126754760742
戲 13.181442260742188
乐 11.974376678466797

热 7.883047580718994
['《', '说', '服', '》', '你', '是', '冰', '我', '是', '热', '[MASK]']
热 11.148796081542969
儿 8.764961242675781
/ 8.298961639404297



In [16]:
TEXT = """
《世界》
[MASK][MASK]不开了
花继续开
"""

CHOICES = []

predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model2)

我 6.648489952087402
['《', '世', '界', '》', '我', '[MASK]', '不', '开', '了', '花', '继', '续', '开']
们 11.359298706054688
也 10.413335800170898
还 9.616384506225586

， 6.5934600830078125
['《', '世', '界', '》', '，', '[MASK]', '不', '开', '了', '花', '继', '续', '开']
爱 9.663169860839844
也 8.148285865783691
永 7.861174583435059

《 6.3722076416015625
['《', '世', '界', '》', '《', '[MASK]', '不', '开', '了', '花', '继', '续', '开']
花 8.496639251708984
爱 8.486017227172852
新 7.992000102996826

------------------------------------------------------------------------------------
大 7.0296831130981445
['《', '世', '界', '》', '大', '[MASK]', '不', '开', '了', '花', '继', '续', '开']
河 9.451949119567871
山 9.290780067443848
海 8.543252944946289

一 6.896337509155273
['《', '世', '界', '》', '一', '[MASK]', '不', '开', '了', '花', '继', '续', '开']
天 10.683612823486328
地 9.901617050170898
口 9.817977905273438

小 6.63884162902832
['《', '世', '界', '》', '小', '[MASK]', '不', '开', '了', '花', '继', '续', '开']
山 10.4845609664917
谜 9.732901573181152
诗 9.433244705200195


In [17]:
TEXT = """
《安慰》
罐头告诉鱼
这里很安全
没有鲨鱼
也没有[MASK][MASK]
"""

CHOICES = []

predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model2)

鲨 9.557595252990723
['《', '安', '慰', '》', '罐', '头', '告', '诉', '鱼', '这', '里', '很', '安', '全', '没', '有', '鲨', '鱼', '也', '没', '有', '鲨', '[MASK]']
鱼 15.059053421020508
牙 7.787806987762451
影 7.576664924621582

鱼 9.086846351623535
['《', '安', '慰', '》', '罐', '头', '告', '诉', '鱼', '这', '里', '很', '安', '全', '没', '有', '鲨', '鱼', '也', '没', '有', '鱼', '[MASK]']
也 8.869028091430664
鱼 8.161286354064941
毒 8.094161033630371

海 8.67890739440918
['《', '安', '慰', '》', '罐', '头', '告', '诉', '鱼', '这', '里', '很', '安', '全', '没', '有', '鲨', '鱼', '也', '没', '有', '海', '[MASK]']
豚 12.03481674194336
底 10.261301040649414
鱼 9.988921165466309

------------------------------------------------------------------------------------
海 9.457998275756836
['《', '安', '慰', '》', '罐', '头', '告', '诉', '鱼', '这', '里', '很', '安', '全', '没', '有', '鲨', '鱼', '也', '没', '有', '海', '[MASK]']
洋 10.992998123168945
底 10.177282333374023
面 9.573942184448242

大 9.067609786987305
['《', '安', '慰', '》', '罐', '头', '告', '诉', '鱼', '这', '里', '很', '安', '全', '没', '有', '鲨'

In [18]:
TEXT = """
《甜甜圈》
生活有时会出现一个大洞
我们可以只看[MASK]的部分
"""
predict_masks(text=TEXT, choices=CHOICES, masks_left=1, num_results=3, model=model2)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=1, num_results=3, model=model2)

它 12.4714937210083
到 11.505626678466797
看 10.755748748779297

------------------------------------------------------------------------------------
它 12.4714937210083
到 11.505626678466797
看 10.755748748779297



In [19]:
TEXT = """
《悲剧》
公主被王子吻醒
发现四周没有别人可以[MASK]
"""

CHOICES = []

predict_masks(text=TEXT, choices=CHOICES, masks_left=1, num_results=NUM_RESULTS, model=model)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=1, num_results=NUM_RESULTS, model=model2)

做 8.110145568847656
用 7.727746486663818
说 6.990058422088623
代 6.6633453369140625
马 6.506006717681885
的 6.326287269592285
学 6.135912895202637
生 6.12570333480835
方 6.1252946853637695
使 6.069259166717529

------------------------------------------------------------------------------------
说 11.365616798400879
进 10.413785934448242
以 10.25168228149414
选 10.208749771118164
生 10.142194747924805
做 9.973831176757812
活 9.749025344848633
自 9.701735496520996
出 9.658236503601074
去 9.648157119750977



In [21]:
TEXT = """
《挫败》
像在大雨中被风吹坏一把伞
有些人学会买坚固的伞
有些人学会[MASK][MASK]不出门
"""

CHOICES = []

predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model)
print("------------------------------------------------------------------------------------")
predict_masks(text=TEXT, choices=CHOICES, masks_left=2, num_results=3, model=model2)

... 7.920332908630371
['《', '挫', '败', '》', '像', '在', '大', '雨', '中', '被', '风', '吹', '坏', '一', '把', '伞', '有', '些', '人', '学', '会', '买', '坚', '固', '的', '伞', '有', '些', '人', '学', '会', '...', '[MASK]', '不', '出', '门']
永 8.29880142211914
三 7.894652843475342
也 7.70860481262207

老 7.33214807510376
['《', '挫', '败', '》', '像', '在', '大', '雨', '中', '被', '风', '吹', '坏', '一', '把', '伞', '有', '些', '人', '学', '会', '买', '坚', '固', '的', '伞', '有', '些', '人', '学', '会', '老', '[MASK]', '不', '出', '门']
年 11.802939414978027
天 10.735246658325195
子 10.58016586303711

不 6.851112365722656
['《', '挫', '败', '》', '像', '在', '大', '雨', '中', '被', '风', '吹', '坏', '一', '把', '伞', '有', '些', '人', '学', '会', '买', '坚', '固', '的', '伞', '有', '些', '人', '学', '会', '不', '[MASK]', '不', '出', '门']
离 10.578926086425781
辞 9.912263870239258
理 9.82224178314209

------------------------------------------------------------------------------------
工 9.829963684082031
['《', '挫', '败', '》', '像', '在', '大', '雨', '中', '被', '风', '吹', '坏', '一', '把', '伞', '有', '些', 